In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

def add_shifts(data, n_in, n_out):
    """featurize by shifting data and creating new columns"""
    df = pd.DataFrame(data)
    cols, names = [], []
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names.append('t-{}'.format(i))
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        names.append('t' if i==0 else 't+{}'.format(i))
    result = pd.concat(cols,axis=1)
    result.columns = names
    result = result.dropna()
    return result

def prepare(data, n_test, n_in, n_out):
    """scale to range [-1,1], add_shifts, split into test & train data"""
    arr = np.array(data.diff()).reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(-1,1))
    arr = add_shifts(scaler.fit_transform(arr), n_in, n_out).values
    return scaler, arr[:-n_test], arr[-n_test:]

def create_lstm(data, n_in, n_batch, n_epochs, n_neurons):
    """split data into train & target, reshape to keras requirements, fit model"""
    X, y = data[:,:n_in], data[:,n_in:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(n_epochs):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

Using TensorFlow backend.


In [2]:
n_test, n_in, n_out = 19, 10, 3
n_epochs, n_neurons, n_batch = 100, 50, 1
src = '../data'
dst = '../models'

files = [f for f in os.listdir(src) if os.path.isfile(os.path.join(src,f))]
for file in files:
    df = pd.read_csv(os.path.join(src,file))
    name, ext = os.path.splitext(file)
    scaler, train, test = prepare(df.price, n_test, n_in, n_out)
    model = create_lstm(train, n_in, n_batch, n_epochs, n_neurons)
    model.save(os.path.join(dst,name+'.h5'))